# Chapter Four - Combining, Reshaping and Aggregating Data

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append("../../../optimus")

In [3]:
from optimus import Optimus
op = Optimus("pandas")

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\LuisA\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


## Concatenating data

In [4]:
df_a = op.create.dataframe({ 
    "id": [143, 225, 545], 
    "name": ["Alice", "Bob", "Charlie"], 
    "city": ["Plymouth", "Bradford", "Norwich"] 
}) 

df_b = op.create.dataframe({ 
    "id": [765, 329, 152], 
    "name": ["Dan", "Erin", "Frank"], 
    "city": ["Bath", "Manchester", "Ripon"] 
}) 

In [5]:
df_a.rows.append(df_b).display("all")

id 1 (int64),name 2 (object),city 3 (object)
143,Alice,Plymouth
225,Bob,Bradford
545,Charlie,Norwich
765,Dan,Bath
329,Erin,Manchester
152,Frank,Ripon


In [6]:
df_a.rows.append(df_b.cols.drop("city")).display("all") 

id 1 (int64),name 2 (object),city 3 (object)
143,Alice,Plymouth
225,Bob,Bradford
545,Charlie,Norwich
765,Dan,nan
329,Erin,nan
152,Frank,nan


In [7]:
df_a.rows.append(df_b.cols.rename("city", "city_b")).display("all") 

id 1 (int64),name 2 (object),city 3 (object),city_b 4 (object)
143,Alice,Plymouth,nan
225,Bob,Bradford,nan
545,Charlie,Norwich,nan
765,Dan,nan,Bath
329,Erin,nan,Manchester
152,Frank,nan,Ripon


## Mapping columns

In [8]:
df_a = op.create.dataframe({ 
    "id": [143, 225, 545], 
    "name": ["Alice", "Bob", "Charlie"], 
    "city": ["Plymouth", "Bradford", "Norwich"] 
})  

df_b = op.create.dataframe({ 
    "id_number": [765, 329, 152], 
    "name": ["Dan", "Erin", "Frank"], 
    "title": ["Bath", "Manchester", "Ripon"] 
}) 

In [9]:
names_map = {
    "id_number": ("id", "id_number"),
    "name": ("name", "name"),
    "city": ("city", "title")
}

df_a.rows.append(df_b, names_map=names_map).display("all")

id_number 1 (int64),name 2 (object),city 3 (object)
143,Alice,Plymouth
225,Bob,Bradford
545,Charlie,Norwich
765,Dan,Bath
329,Erin,Manchester
152,Frank,Ripon


## Concatenating columns

In [10]:
df_a = op.create.dataframe({
    "id": [143, 225, 545],
    "name": ["Alice", "Bob", "Charlie"],
    "city": ["Plymouth", "Bradford", "Norwich"]
})

df_b = op.create.dataframe({
    "age": [25, 35, 45],
    "placeholder": ["foo", "bar", "baz"]
})

In [11]:
df_a.cols.concat(df_b)

id 1 (int64),name 2 (object),city 3 (object),age 4 (int64),placeholder 5 (object)
143,Alice,Plymouth,25,foo
225,Bob,Bradford,35,bar
545,Charlie,Norwich,45,baz


In [12]:
df_a.cols.select(["id", "name"]).cols.concat(df_b.cols.select([ "age"]))

id 1 (int64),name 2 (object),age 3 (int64)
143,Alice,25
225,Bob,35
545,Charlie,45


## Joining data

In [13]:
df_a = op.create.dataframe({
    "id": [143, 225, 545, 765, 152],
    "name": ["Alice", "Bob", "Charlie", "Dan", "Frank"]
})

df_b = op.create.dataframe({
    "id": [225, 545, 765, 152, 329],
    "city": ["Bradford", "Norwich", "Bath", "Ripon", "Manchester"],
    "placeholder": ["BRA", "NOR", "BAT", "RIP", "MAN"]
})

In [14]:
df_a.join(df_b, how="outer", on="id")

id 1 (object),name 2 (object),city 3 (object),placeholder 4 (object)
143,Alice,nan,nan
225,Bob,Bradford,BRA
545,Charlie,Norwich,NOR
765,Dan,Bath,BAT
152,Frank,Ripon,RIP
329,nan,Manchester,MAN


In [15]:
df_a.join(df_b, on="id")

id 1 (object),name 2 (object),city 3 (object),placeholder 4 (object)
143,Alice,nan,nan
225,Bob,Bradford,BRA
545,Charlie,Norwich,NOR
765,Dan,Bath,BAT
152,Frank,Ripon,RIP


In [16]:
df_a.join(df_b, on="id", how="inner")

id 1 (object),name 2 (object),city 3 (object),placeholder 4 (object)
225,Bob,Bradford,BRA
545,Charlie,Norwich,NOR
765,Dan,Bath,BAT
152,Frank,Ripon,RIP


## Reshaping and Pivoting

In [17]:
df = op.create.dataframe({
    "date": ["1/1/21", "1/1/21", "1/2/21", "1/2/21", "1/3/21", "1/3/21", "1/3/21", "1/3/21", "1/3/21"],
    "product": ["Coffee", "Coffee", "Tea", "Coffee", "Tea", "Coffee", "Tea", "Tea", "Coffee"],
    "size": ["big", "big", "big", "big", "big", "small", "small", "small", "small"],
    "price": [1.5, 1.5, 2, 1.5, 2, 1, 1.25, 1.25, 1]
})
df

date 1 (object),product 2 (object),size 3 (object),price 4 (float64)
1/1/21,Coffee,big,1.5
1/1/21,Coffee,big,1.5
1/2/21,Tea,big,2.0
1/2/21,Coffee,big,1.5
1/3/21,Tea,big,2.0
1/3/21,Coffee,small,1.0
1/3/21,Tea,small,1.25
1/3/21,Tea,small,1.25
1/3/21,Coffee,small,1.0


### Pivoting

In [18]:
df.pivot("date", groupby="product")

product 1 (object),1/1/21 2 (float64),1/2/21 3 (float64),1/3/21 4 (float64)
Coffee,2.0,1.0,2.0
Tea,nan,1.0,3.0


In [19]:
df.pivot("date", groupby="product", agg=("count", "size"))

product 1 (object),1/1/21 2 (float64),1/2/21 3 (float64),1/3/21 4 (float64)
Coffee,2.0,1.0,2.0
Tea,nan,1.0,3.0


In [20]:
df.pivot("date", groupby="product", agg=("mean", "price"))

product 1 (object),1/1/21 2 (float64),1/2/21 3 (float64),1/3/21 4 (float64)
Coffee,1.5,1.5,1.0
Tea,nan,2.0,1.5


In [21]:
df.pivot("date", groupby="product", values="size")

product 1 (object),1/1/21 2 (object),1/2/21 3 (object),1/3/21 4 (object)
Coffee,big,big,small
Tea,nan,big,big


### Stacking

In [22]:
df = op.create.dataframe({
    "product": ["Coffee", "Coffee", "Tea", "Tea"],
    "size": ["big", "small", "big", "small"],
    "price": [1.5, 1, 2, 1.25],
    "cost": [0.24, 0.2, 0.32, 0.3]
})
df

product 1 (object),size 2 (object),price 3 (float64),cost 4 (float64)
Coffee,big,1.5,0.24
Coffee,small,1.0,0.2
Tea,big,2.0,0.32
Tea,small,1.25,0.3


In [23]:
df = df.stack(index=["product", "size"])
df

product 1 (object),size 2 (object),variable 3 (object),value 4 (float64)
Coffee,big,price,1.5
Coffee,big,cost,0.24
Coffee,small,price,1.0
Coffee,small,cost,0.2
Tea,big,price,2.0
Tea,big,cost,0.32
Tea,small,price,1.25
Tea,small,cost,0.3


### Unstack

In [24]:
df.unstack(index=["product", "size", "variable"])

product 1 (object),size 2 (object),cost 3 (float64),price 4 (float64)
Coffee,big,0.24,1.5
Coffee,small,0.2,1.0
Tea,big,0.32,2.0
Tea,small,0.3,1.25


In [25]:
df.unstack(index=["product", "size", "variable"], level=1)

product 1 (object),variable 2 (object),big 3 (float64),small 4 (float64)
Coffee,cost,0.24,0.2
Coffee,price,1.5,1.0
Tea,cost,0.32,0.3
Tea,price,2.0,1.25


In [26]:
df.unstack(index=["product", "size", "variable"], level=-2)

product 1 (object),variable 2 (object),big 3 (float64),small 4 (float64)
Coffee,cost,0.24,0.2
Coffee,price,1.5,1.0
Tea,cost,0.32,0.3
Tea,price,2.0,1.25


In [27]:
df.unstack(index=["product", "size", "variable"], level="size")

product 1 (object),variable 2 (object),big 3 (float64),small 4 (float64)
Coffee,cost,0.24,0.2
Coffee,price,1.5,1.0
Tea,cost,0.32,0.3
Tea,price,2.0,1.25


### Melt

In [28]:
df = op.create.dataframe({
    "product": ["Coffee", "Coffee", "Tea", "Tea"],
    "size": ["big", "small", "big", "small"],
    "price": [1.5, 1, 2, 1.25],
    "cost": [0.24, 0.2, 0.32, 0.3]
})
df

product 1 (object),size 2 (object),price 3 (float64),cost 4 (float64)
Coffee,big,1.5,0.24
Coffee,small,1.0,0.2
Tea,big,2.0,0.32
Tea,small,1.25,0.3


In [29]:
df.melt(id_cols=["product","size"])

product 1 (object),size 2 (object),variable 3 (object),value 4 (float64)
Coffee,big,price,1.5
Coffee,small,price,1.0
Tea,big,price,2.0
Tea,small,price,1.25
Coffee,big,cost,0.24
Coffee,small,cost,0.2
Tea,big,cost,0.32
Tea,small,cost,0.3


In [30]:
df.melt(id_cols=["product","size"], value_cols=["price", "cost"])

product 1 (object),size 2 (object),variable 3 (object),value 4 (float64)
Coffee,big,price,1.5
Coffee,small,price,1.0
Tea,big,price,2.0
Tea,small,price,1.25
Coffee,big,cost,0.24
Coffee,small,cost,0.2
Tea,big,cost,0.32
Tea,small,cost,0.3


In [31]:
df.melt(id_cols=["product", "size"], var_name="foo", value_name="bar")

product 1 (object),size 2 (object),foo 3 (object),bar 4 (float64)
Coffee,big,price,1.5
Coffee,small,price,1.0
Tea,big,price,2.0
Tea,small,price,1.25
Coffee,big,cost,0.24
Coffee,small,cost,0.2
Tea,big,cost,0.32
Tea,small,cost,0.3


## Aggregations

In [32]:
df = op.load.file("foo.csv")
df

name 1 (object),job 2 (object),id 3 (int64)
optimus,Leader,1
optimus,Espionage,2
bumblebee,1,3
bumblebee,3,4


In [33]:
df.cols.min("id")

1

In [34]:
df.agg({"id": "min"})

1

In [35]:
df.agg({"name": "min", "id": "max"})

{'name_min': 'bumblebee', 'id_max': 4}

In [36]:
print(df.cols.std("id"))
print(df.cols.min("id"))

1.2909944487358056
1


In [37]:
df.agg({"id": "std", "name": "min"})

{'id_std': 1.2909944487358056, 'name_min': 'bumblebee'}

### Aggregating and Grouping

In [38]:
df.agg({"id": "min"}, groupby="name")

{'bumblebee': 3, 'optimus': 1}

In [39]:
df.agg({"id": "min"}, groupby="name")["bumblebee"]

3